# PDFView

> Preview function for PDF files in Ragfood

In [ ]:
#| default_exp pdfview

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import PyPDF2, json, os, subprocess, io, tempfile
from ipywidgets import HTML, Layout, Image as IPyImage, VBox, HBox, Button
from pathlib import Path
from pdf2image import convert_from_path
from Ragfood.mediator import Colleague
from Ragfood.states import *

In [ ]:
#| export
class PDFView(Colleague):
    """Preview function for PDF files."""
    
    def __init__(self, mediator=None):
        """Initialize PDFView widget."""
        # attributes and super constructor
        super().__init__(mediator)
        self.pdf_title, self.pdf_path, self.num_pages, self.pdf_author, self.thumbnails = '', None, 0, '', []
        
        # widgets
        self.info_hm = HTML()
        self.thumbs_hb = HBox(layout=Layout(width='100%', height='450px', overflow='auto'))
        self.thumbNext_bu = Button(description='>>>', layout={'width': 'auto'}, disabled=True)
        self.thumbBack_bu = Button(description='<<<', layout={'width': 'auto'}, disabled=True)
        self.widget = VBox(children=[self.info_hm, self.thumbs_hb, HBox(children=[self.thumbBack_bu, self.thumbNext_bu])])
        self.displayInfo()
        self.changed('OnPDFViewCreated', RAGFOOD_OK)
        
        # events
        self.thumbNext_bu.on_click(self.onThumbNavigate)
        self.thumbBack_bu.on_click(self.onThumbNavigate)
    
    def onThumbNavigate(self, b):
        """Handle thumbnail navigation."""
        if b.description == ">>>":
            self.display_thumbnails(first_page=self.first_page+5)
        elif b.description == "<<<":
            self.display_thumbnails(first_page=self.first_page-5)
    
    def displayInfo(self):
        """Display PDF information."""
        self.info_hm.value = f"""
        <div style="background-color: #f0f8ff; padding: 1px"><hr>
          <p><strong>📄 Document:</strong> {self.pdf_title}</p>
          <p><strong>📊 Pages:</strong> {self.num_pages} - <strong>✍️ Author(s):</strong> {self.pdf_author}</p>
        </div>"""
    
    def setPDF(self, path):
        """Set PDF file to display."""
        self.pdf_path = path
        
        # check path exists
        if not os.path.exists(path):
            self.changed('OnPDFViewError', RAGFOOD_PDF_NOT_AVAILABLE, f"PDF file not found: {path}")
            return
        
        # Extract metadata from PDF file
        try:
            with open(path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                
                # Number of pages
                self.num_pages = len(pdf_reader.pages)
                
                # Extract metadata
                if pdf_reader.metadata:
                    self.pdf_title = pdf_reader.metadata.title if pdf_reader.metadata.title else Path(path).stem
                    self.pdf_author = pdf_reader.metadata.author if pdf_reader.metadata.author else 'Unknown Author'
            
            # thumbnails
            self.display_thumbnails()
            
        except Exception as e:
            self.pdf_title = Path(path).stem
            self.pdf_author = "Unknown Author"
            self.changed('OnPDFViewError', RAGFOOD_PDF_ERROR_READING_METADATA, f"Error reading metadata: {e}")
        
        finally:
            self.displayInfo()
    
    def display_thumbnails(self, num_thumbs=5, first_page=1, dpi=150):
        """Generate thumbnail images from PDF"""
        try:
            # Convert pages to images
            if first_page < 1 or first_page > self.num_pages:
                first_page = 1
            if first_page + num_thumbs > self.num_pages:
                num_thumbs = self.num_pages - first_page
            self.first_page = first_page
            
            self.thumbnails = convert_from_path(self.pdf_path, dpi=dpi, first_page=first_page, last_page=min((first_page-1)+num_thumbs, self.num_pages))
            
        except Exception as e:
            self.thumbnails = []
            self.changed('OnPDFViewError', RAGFOOD_PDF_NOT_AVAILABLE, f"Error generating thumbnails: {e}")
            return []
        
        # Create thumbnail widgets
        thumbnail_widgets = []
        
        for i, img in enumerate(self.thumbnails):
            # Convert PIL image to bytes for display
            img_bytes = io.BytesIO()
            # Resize for display
            display_img = img.copy()
            display_img.thumbnail((300, 400))
            display_img.save(img_bytes, format='PNG')
            
            # Create image widget
            img_widget = IPyImage(value=img_bytes.getvalue(), format='png',
                                 layout={'margin': '3px', 'border': '1px solid #ccc'})
            
            # Create label
            label_widget = HTML(f"<center><small>Page {i+first_page}</small></center>")
            
            # Combine in VBox
            page_widget = VBox([img_widget, label_widget])
            thumbnail_widgets.append(page_widget)
        
        # Display in horizontal layout
        self.thumbs_hb.children = tuple(thumbnail_widgets)
        
        # enable navigation
        self.thumbNext_bu.disabled = False
        self.thumbBack_bu.disabled = False

## Example Usage

In [ ]:
# Example usage (requires pdf2image and PyPDF2)
# import tempfile
# import os
# 
# # Download sample PDF
# os.system(f"""[ -f {tempfile.gettempdir()}/mathematics_for_machine_learning.pdf ] || curl -o {tempfile.gettempdir()}/mathematics_for_machine_learning.pdf https://mml-book.github.io/book/mml-book.pdf""")
# 
# # Create and show PDFView
# pdfview = PDFView()
# display(pdfview.widget)
# pdfview.setPDF(f"{tempfile.gettempdir()}/mathematics_for_machine_learning.pdf")